In [1]:
import os
import sys
import pandas as pd
import numpy as np

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , Dense, LSTM


In [3]:
file_path = "C:\\Users\\graykar\\Downloads\\Pyhton\\Data Files\\spa.txt"

In [4]:
with open(file_path , 'r', encoding='UTF-8') as f:
    lines = f.read().split('\n')

In [5]:
latent_dimen = 256
ephoc = 30
num_samples = 6000 # number of samples to train
batch_size = 64

In [6]:
input_texts = []
target_texts = []
input_char = set()
target_char = set()

for line in lines[: 10000]: # I am considering only 10000 samples from 122550 records
    input_text , target_text , _ = line.split('\t')
    target_text = '\t' + target_text + '\n'  # using first word as \t and End of Sequece as \n
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_char:
            input_char.add(char)
    for char in target_text:
        if char not in target_char:
            target_char.add(char)

In [7]:
input_chars = sorted(list(input_char))
target_chars = sorted(list(target_char))
num_encoder_token = len(input_chars)
num_decoder_token = len(target_chars)
max_encoder_seq_len = max([len(i) for i in input_texts])
max_decoder_seq_len = max([len(i) for i in target_texts])

In [8]:
print("num_encoder_token :" , num_encoder_token)
print("num_decoder_token :" , num_decoder_token)
print("max_encoder_seq_len :" , max_encoder_seq_len)
print("max_decoder_seq_len :" , max_decoder_seq_len)


num_encoder_token : 69
num_decoder_token : 84
max_encoder_seq_len : 16
max_decoder_seq_len : 42


In [9]:
input_token_index = dict([(index, char) for char ,index in enumerate(input_chars)])
target_token_index = dict([(index, char) for char ,index in enumerate(target_chars)])

In [10]:
encoder_input_data = np.zeros( (len(input_texts) , max_encoder_seq_len , num_encoder_token ) , dtype='float32' )

decoder_input_data = np.zeros( (len(input_texts) , max_decoder_seq_len , num_decoder_token ) , dtype='float32' )

decoder_target_data = np.zeros( (len(input_texts) , max_decoder_seq_len , num_decoder_token ) , dtype='float32' )

In [11]:
encoder_input_data.shape, decoder_input_data.shape , decoder_target_data.shape

((10000, 16, 69), (10000, 42, 84), (10000, 42, 84))

### One - Hot represnetation of data (English) in 3D array

In [12]:
for i , (input_text, target_text) in enumerate(zip(input_texts,target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1.
    encoder_input_data[i , t+1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]] = 1.
    
        if t>0:                              # Decoder target will be ahead of one timestamp 
            decoder_target_data[i, t-1, target_token_index[char]] = 1.
            
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t: , target_token_index[' ']] = 1.

In [13]:
encoder_inputs = Input(shape=(None, num_encoder_token))
encoder = LSTM(latent_dimen,return_state=True)
encoder_output , state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [14]:
decoder_inputs = Input(shape=(None , num_decoder_token))
deocder_lstm = LSTM(latent_dimen,return_sequences=True , return_state=True )
decoder_output , _ ,_ = deocder_lstm(decoder_inputs, initial_state=encoder_states)

In [15]:
decoder_dense = Dense(num_decoder_token, activation='softmax' )
decoder_outputs = decoder_dense(decoder_output)

In [16]:
model = Model([encoder_inputs ,decoder_inputs] , decoder_outputs )
model.compile(optimizer='rmsprop' , loss= 'categorical_crossentropy',metrics=['accuracy'])

In [17]:
model.fit([encoder_input_data , decoder_input_data] , decoder_target_data , 
          batch_size=batch_size , 
          epochs=ephoc, 
          validation_split=0.2
         )

Train on 8000 samples, validate on 2000 samples
Epoch 1/30
8000/8000 [==============================] - 62s 8ms/sample - loss: 1.3977 - accuracy: 0.6612 - val_loss: 1.2994 - val_accuracy: 0.6440
Epoch 2/30
8000/8000 [==============================] - 52s 6ms/sample - loss: 1.0308 - accuracy: 0.7195 - val_loss: 1.0786 - val_accuracy: 0.6923
Epoch 3/30
8000/8000 [==============================] - 66s 8ms/sample - loss: 0.8610 - accuracy: 0.7506 - val_loss: 0.9461 - val_accuracy: 0.7177
Epoch 4/30
8000/8000 [==============================] - 72s 9ms/sample - loss: 0.7741 - accuracy: 0.7689 - val_loss: 0.8806 - val_accuracy: 0.7366
Epoch 5/30
8000/8000 [==============================] - 41s 5ms/sample - loss: 0.7166 - accuracy: 0.7843 - val_loss: 0.8270 - val_accuracy: 0.7526
Epoch 6/30
8000/8000 [==============================] - 62s 8ms/sample - loss: 0.6740 - accuracy: 0.7960 - val_loss: 0.7921 - val_accuracy: 0.7579
Epoch 7/30
8000/8000 [==============================] - 67s 8ms/sample

In [18]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dimen,))
decoder_state_input_c = Input(shape=(latent_dimen,))
decoder_states_input = [ decoder_state_input_h , decoder_state_input_c ]

decoder_outputs , state_h , state_c = deocder_lstm(decoder_inputs, initial_state=decoder_states_input)

decoder_state = [state_h , state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model( 
                    [decoder_inputs] + decoder_states_input, 
                    [decoder_outputs] + decoder_state )

reverse_input_char_index = dict( (i, char) for char, i in input_token_index.items() )
reverse_target_char_index = dict( (i, char) for char, i in target_token_index.items() )


In [19]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1,1,num_decoder_token))
    
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value )
        
        sampled_token_index = np.argmax(output_tokens[0 ,-1 , :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True
            
        taeget_seq = np.zeros((1, 1, num_decoder_token))
        target_seq[0, 0, sampled_token_index] = 1.
        
        states_value = [h, c]
    
    return decoded_sentence

In [20]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_seq = decode_sequence(input_seq) 
    
    print('-')
    print('Input Sentense:' , input_texts[seq_index] )
    print('Decoded Sentense:' , decoded_seq )


-
Input Sentense: Go.
Decoded Sentense: Veya.

-
Input Sentense: Go.
Decoded Sentense: Veya.

-
Input Sentense: Go.
Decoded Sentense: Veya.

-
Input Sentense: Go.
Decoded Sentense: Veya.

-
Input Sentense: Hi.
Decoded Sentense: Soy yyyyyyyyydudyyyyyyyyyyyyyyyyyyyyyyyyyyy
-
Input Sentense: Run!
Decoded Sentense: ¡Corrá!!!!

-
Input Sentense: Run!
Decoded Sentense: ¡Corrá!!!!

-
Input Sentense: Run!
Decoded Sentense: ¡Corrá!!!!

-
Input Sentense: Run!
Decoded Sentense: ¡Corrá!!!!

-
Input Sentense: Run.
Decoded Sentense: Corrndió...................................
-
Input Sentense: Who?
Decoded Sentense: ¿Quiééééénéééééddééééiiiiiiiiiiiiiiiiiiiiii
-
Input Sentense: Wow!
Decoded Sentense: ¡EnEccuccchhiccstééééannntttttttttttttttnnn
-
Input Sentense: Fire!
Decoded Sentense: ¡Fuerrndizzzzzía!!!!nnnnnnnnnnnnnnn

-
Input Sentense: Fire!
Decoded Sentense: ¡Fuerrndizzzzzía!!!!nnnnnnnnnnnnnnn

-
Input Sentense: Fire!
Decoded Sentense: ¡Fuerrndizzzzzía!!!!nnnnnnnnnnnnnnn

-
Input Sentense: Help!
